<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [9]:
lista_terminos = list(np.char.split(corpus, " "))
print(f"Documentos como listas de términos: {lista_terminos}")
print()
no_repetidos = np.unique(np.concatenate((lista_terminos)))
print(f"Vector de términos no repetidos: {no_repetidos}")

Documentos como listas de términos: [['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]

Vector de términos no repetidos: ['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [14]:
import pandas as pd
one_hot = np.array([np.isin(no_repetidos, terminos).astype(int) for terminos in lista_terminos])
df = pd.DataFrame(one_hot, columns=no_repetidos)
df.rename(index={0: 'Doc1', 1: 'Doc2', 2: 'Doc3'}, inplace=True)
df

,de,dia,el,es,gracias,hoy,martes,muchas,que
Doc1,0,1,0,1,0,1,0,0,1
Doc2,1,1,1,1,0,1,1,0,0
Doc3,0,0,0,0,1,0,1,1,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [15]:
freq = np.array([[terminos.count(termino) for termino in no_repetidos] for terminos in lista_terminos])
df = pd.DataFrame(freq, columns=no_repetidos)
df.rename(index={0: 'Doc1', 1: 'Doc2', 2: 'Doc3'}, inplace=True)
df

,de,dia,el,es,gracias,hoy,martes,muchas,que
Doc1,0,1,0,1,0,1,0,0,1
Doc2,1,1,1,1,0,1,2,0,0
Doc3,0,0,0,0,1,0,1,1,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [37]:
tf_matrix = freq

idf_matrix = np.log10(len(corpus) / np.array(one_hot).sum(axis=0))

tfidf_matrix = tf_matrix * idf_matrix

df = pd.DataFrame(tfidf_matrix, columns=no_repetidos)
df

,de,dia,el,es,gracias,hoy,martes,muchas,que
0,0.000000,0.176091,0.000000,0.176091,0.000000,0.176091,0.000000,0.000000,0.477121
1,0.477121,0.176091,0.477121,0.176091,0.000000,0.176091,0.352183,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.477121,0.000000,0.176091,0.477121,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [43]:
def retrieve_similar_documents(corpus, idx):
    lista_terminos = list(np.char.split(corpus, " "))
    no_repetidos = np.unique(np.concatenate(lista_terminos))
    one_hot = np.array([np.isin(no_repetidos, terminos).astype(int) for terminos in lista_terminos])
    freq = np.array([[terminos.count(termino) for termino in no_repetidos] for terminos in lista_terminos])
    tf_matrix = freq
    idf_matrix = np.log10(len(corpus) / np.array(one_hot).sum(axis=0))
    tfidf_matrix = tf_matrix * idf_matrix
    
    similitudes = [cosine_similarity(tfidf_matrix[idx], tfidf_matrix[i]) for i in range(len(corpus))]
    sorted_indices = np.argsort(similitudes)[::-1]
    
    return sorted_indices

corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

doc_index = 2
print(retrieve_similar_documents(corpus, doc_index))


[2 1 0]
